In [1]:
import numpy as np # linear algebra
import seaborn as sns
sns.set(style='whitegrid')
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import h5py

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_dataset = h5py.File('E:/FCI/year 4/second term/NN/labs/neural-network-and-deep-learning-master/lab3/train_catvnoncat.hdf5', "r")
train_set_x = np.array(train_dataset["train_set_x"][:]).astype(np.float32) # your train set features
train_set_y = np.array(train_dataset["train_set_y"][:]).astype(np.float32) # your train set labels

test_dataset = h5py.File('E:/FCI/year 4/second term/NN/labs/neural-network-and-deep-learning-master/lab3/test_catvnoncat.hdf5', "r")
test_set_x= np.array(test_dataset["test_set_x"][:]).astype(np.float32) # your test set features
test_set_y = np.array(test_dataset["test_set_y"][:]).astype(np.float32) # your test set labels
classes = np.array(test_dataset["list_classes"][:]) # the list of classes
data_size = train_set_x.shape[0]



In [3]:

#print dataset shapes(dimensions)
print("train set x dim= "+str(train_set_x.shape))
print("train set y dim= "+str(train_set_y.shape))
print("test set x dim= "+str(test_set_x.shape))
print("test set y dim= "+str(test_set_y.shape))
print("classes = "+str(classes.shape))


train set x dim= (209, 64, 64, 3)
train set y dim= (209,)
test set x dim= (50, 64, 64, 3)
test set y dim= (50,)
classes = (2,)


In [4]:
#reshape dataset
#train data
train_set_x = train_set_x.reshape(train_set_x.shape[0],-1)
train_set_y = train_set_y.reshape(1,train_set_y.shape[0]).T
    
#test data
test_set_x = test_set_x.reshape(test_set_x.shape[0],-1)
test_set_y = test_set_y.reshape(1,test_set_y.shape[0]).T

train_set_y_n = np.where(train_set_y == 0, 1, 0)
train_set_y =  np.c_[train_set_y,train_set_y_n]

test_set_y_n = np.where(test_set_y == 0, 1, 0)
test_set_y =  np.c_[test_set_y,test_set_y_n]
print(test_set_y.shape)
print(train_set_y.shape)


(50, 2)
(209, 2)


In [5]:

#print dataset shapes(dimensions)
print("train set x dim= "+str(train_set_x.shape))
print("train set y dim= "+str(train_set_y.shape))
print("test set x dim= "+str(test_set_x.shape))
print("test set y dim= "+str(test_set_y.shape))
print("data_size = "+str(data_size))


train set x dim= (209, 12288)
train set y dim= (209, 2)
test set x dim= (50, 12288)
test set y dim= (50, 2)
data_size = 209


In [6]:
#normalize data
train_set_x=train_set_x/255
test_set_x=test_set_x/255

In [7]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 209
display_step = 1
num_nodes_hl1=100
num_nodes_hl2=100
num_nodes_hl3=100
num_nodes_hl4=100
num_classes =classes.shape[0]

In [11]:
def NN_model(data):
    
    
    hidden_layer_1={'weights':tf.Variable(tf.random_normal([train_set_x.shape[1],num_nodes_hl1])),
         'biases':tf.Variable(tf.random_normal([num_nodes_hl1]))}
    
    hidden_layer_2={'weights':tf.Variable(tf.random_normal([num_nodes_hl1,num_nodes_hl2])),
         'biases':tf.Variable(tf.random_normal([num_nodes_hl2]))}
    
    hidden_layer_3={'weights':tf.Variable(tf.random_normal([num_nodes_hl2,num_nodes_hl3])),
         'biases':tf.Variable(tf.random_normal([num_nodes_hl3]))}
    
    hidden_layer_4={'weights':tf.Variable(tf.random_normal([num_nodes_hl3,num_nodes_hl4])),
         'biases':tf.Variable(tf.random_normal([num_nodes_hl4]))}
    
    output_layer={'weights':tf.Variable(tf.random_normal([num_nodes_hl4,train_set_y.shape[1]])),
         'biases':tf.Variable(tf.random_normal([train_set_y.shape[1]]))}

    
    l1= tf.add(tf.matmul(data, hidden_layer_1['weights']) ,hidden_layer_1['biases'])
    l1= tf.nn.sigmoid(l1)
    
    l2= tf.add(tf.matmul(l1, hidden_layer_2['weights']) ,hidden_layer_2['biases'])
    l2= tf.nn.sigmoid(l2)
    
    l3= tf.add(tf.matmul(l2, hidden_layer_3['weights']) ,hidden_layer_3['biases'])
    l3= tf.nn.sigmoid(l3)
    
    l4= tf.add(tf.matmul(l3, hidden_layer_4['weights']) ,hidden_layer_4['biases'])
    l4= tf.nn.sigmoid(l4)
    
    output= tf.add(tf.matmul(l4, output_layer['weights']) ,output_layer['biases'])
    output= tf.nn.sigmoid(output)
    
    return output

In [12]:
def trainModel():
    x = tf.placeholder(tf.float32, [None, train_set_x.shape[1]])
    y_ = tf.placeholder(tf.float32, [None, train_set_y.shape[1]])

    #y_ = np.array([None, train_set_y.shape[1]]).astype(np.float32) # your train set labels
    prediction=NN_model(train_set_x)
     # calculating cost
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=train_set_y, logits=prediction)
    cost=tf.reduce_mean(cross_entropy)
    train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    #train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Training cycle
        for epoch in range(training_epochs):
            loss = 0.
            total_batch = int(data_size/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                pos = (i % int(data_size / batch_size)) * batch_size
                batch_xs = train_set_x[pos:pos+batch_size]
                batch_ys = train_set_y[pos:pos+batch_size]
                # Run optimization op (backprop) and cost op (to get loss value)

                _,c = sess.run([train_step,cost], feed_dict={x: batch_xs, y_: batch_ys})
                loss +=c
            print('epoch',epoch,'out of',training_epochs,'loss :',loss)
            
        correct_prediction = tf.equal(tf.argmax(train_set_y, 1), tf.argmax(prediction, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print('train accuracy %g' % accuracy.eval(feed_dict={x: train_set_x, y_: train_set_y}))
        print('test accuracy %g' % accuracy.eval(feed_dict={x: test_set_x, y_: test_set_y}))


In [13]:
trainModel()

epoch 0 out of 25 loss : 0.6728259921073914
epoch 1 out of 25 loss : 0.6572860479354858
epoch 2 out of 25 loss : 0.6567369699478149
epoch 3 out of 25 loss : 0.6556827425956726
epoch 4 out of 25 loss : 0.6547220945358276
epoch 5 out of 25 loss : 0.6434725522994995
epoch 6 out of 25 loss : 0.6312008500099182
epoch 7 out of 25 loss : 0.617596447467804
epoch 8 out of 25 loss : 0.6204277873039246
epoch 9 out of 25 loss : 0.6032322645187378
epoch 10 out of 25 loss : 0.617781937122345
epoch 11 out of 25 loss : 0.5964213013648987
epoch 12 out of 25 loss : 0.598660945892334
epoch 13 out of 25 loss : 0.6097392439842224
epoch 14 out of 25 loss : 0.6102266907691956
epoch 15 out of 25 loss : 0.5875270366668701
epoch 16 out of 25 loss : 0.6074577569961548
epoch 17 out of 25 loss : 0.5857971906661987
epoch 18 out of 25 loss : 0.6049932241439819
epoch 19 out of 25 loss : 0.5834990739822388
epoch 20 out of 25 loss : 0.5968355536460876
epoch 21 out of 25 loss : 0.5773413777351379
epoch 22 out of 25 loss